# Artificial Neural Network
This is for the Deep Learning A-Z course on Udemy. All copyright belongs to the creator.

In this Artificial Neural Network, we are using Keras to build a simple neural network trained with bank data to predict whether the new customer will eventually leave the bank.

## 1. Data Preprocessing


In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Importing the dataset
dataset = pd.read_csv('Churn_modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [3]:
# Encoding categorical data
# Encoding the Independent Variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:,1:]

In [4]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [5]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## 2. Build and train ANN

In [6]:
# Import Keras libraries
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [7]:
# Defining NN as sequence of layers
# Initializing the ANN 
classifier = Sequential()

In [8]:
# Add first input layer and first hidden layer
# Initialze weights and input data
# Use Rec Activation function
# Add function to add layers
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 2s 295us/step - loss: 0.4844 - acc: 0.7945
Epoch 2/100
8000/8000 [==============================] - 2s 278us/step - loss: 0.4267 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 2s 282us/step - loss: 0.4196 - acc: 0.8142
Epoch 4/100
8000/8000 [==============================] - 2s 281us/step - loss: 0.4137 - acc: 0.8292
Epoch 5/100
8000/8000 [==============================] - 2s 283us/step - loss: 0.4093 - acc: 0.8330
Epoch 6/100
8000/8000 [==============================] - 2s 286us/step - loss: 0.4064 - acc: 0.8327
Epoch 7/100
8000/8000 [==============================] - 2s 289us/step - loss: 0.4046 - acc: 0.8344
Epoch 8/100
8000/8000 [==============================] - 2s 284us/step - loss: 0.4027 - acc: 0.8344
Epoch 9/100
8000/8000 [==============================] - 2s 283us/step - loss: 0.4014 - acc: 0.8347
Epoch 10/100
8000/8000 [==============================] - 2s 286us/step - loss: 0.3999 - acc: 0.8359

8000/8000 [==============================] - 2s 282us/step - loss: 0.3363 - acc: 0.8644
Epoch 83/100
8000/8000 [==============================] - 2s 280us/step - loss: 0.3356 - acc: 0.8646
Epoch 84/100
8000/8000 [==============================] - 2s 277us/step - loss: 0.3359 - acc: 0.8647
Epoch 85/100
8000/8000 [==============================] - 2s 280us/step - loss: 0.3359 - acc: 0.8649
Epoch 86/100
8000/8000 [==============================] - 2s 281us/step - loss: 0.3356 - acc: 0.8646
Epoch 87/100
8000/8000 [==============================] - 2s 277us/step - loss: 0.3361 - acc: 0.8634
Epoch 88/100
8000/8000 [==============================] - 2s 273us/step - loss: 0.3355 - acc: 0.8641
Epoch 89/100
8000/8000 [==============================] - 2s 280us/step - loss: 0.3351 - acc: 0.8635
Epoch 90/100
8000/8000 [==============================] - 2s 272us/step - loss: 0.3354 - acc: 0.8631
Epoch 91/100
8000/8000 [==============================] - 2s 288us/step - loss: 0.3347 - acc: 0.8645
Epo

In [9]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5) # Separate all probability to true or false

In [10]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1513,   82],
       [ 187,  218]])

In [11]:
# Accuracy 
print("The accuracy of the predicator is",(1513 + 215)/ 2000) 

The accuracy of the predicator is 0.864


In [12]:
# Sample new test
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = (new_prediction > 0.5)

new_prediction

array([[False]], dtype=bool)

## 3. Evaluating ANN

In [7]:
# Rebuild ANN with K-fold validation
# Import tools
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [8]:
# Make a classifier function for Keras to take as input
def build_classifier():
    # Same code from before
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [9]:
Keras_classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator = Keras_classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)

Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
7200/7200 [==============================] - 4s 509us/step - loss: 0.4877 - acc: 0.7949
Epoch 2/100
7200/7200 [==============================] - 4s 508us/step - loss: 0.4869 - acc: 0.7962
Epoch 2/100
7200/7200 [==============================] - 4s 506us/step - loss: 0.4852 - acc: 0.7971
Epoch 2/100
7200/7200 [==============================] - 4s 523us/step - loss: 0.4868 - acc: 0.7967
Epoch 2/100
7200/7200 [==============================] - 3s 416us/step - loss: 0.4261 - acc: 0.7956
Epoch 3/100
7200/7200 [==============================] - 3s 417us/step - loss: 0.4271 - acc: 0.7971
Epoch 3/100
7200/7200 [==============================] - 3s 432us/step - loss: 0.4271 - acc: 0.7967
Epoch 3/100
7200/7200 [==============================] - 3s 435us/step - loss: 0.4251 - acc: 0.7975
Epoch 3/100
7200/7200 [==============================] - 3s 430us/step - loss: 0.4188 - acc: 0.8100
Epoch 4/100
7200/7200 [==============================] - 3s 443u

7200/7200 [==============================] - 3s 449us/step - loss: 0.3941 - acc: 0.8369
Epoch 19/100
7200/7200 [==============================] - 3s 434us/step - loss: 0.3984 - acc: 0.8342
Epoch 19/100
7200/7200 [==============================] - 3s 455us/step - loss: 0.4057 - acc: 0.8331
Epoch 19/100
7200/7200 [==============================] - 3s 423us/step - loss: 0.3935 - acc: 0.8386
Epoch 20/100
7200/7200 [==============================] - 3s 428us/step - loss: 0.3973 - acc: 0.8375
Epoch 20/100
7200/7200 [==============================] - 3s 410us/step - loss: 0.4051 - acc: 0.8351
Epoch 20/100
7200/7200 [==============================] - 3s 389us/step - loss: 0.3976 - acc: 0.8347
Epoch 21/100
7200/7200 [==============================] - 3s 404us/step - loss: 0.3974 - acc: 0.8346
Epoch 21/100
7200/7200 [==============================] - 3s 410us/step - loss: 0.3937 - acc: 0.8375
Epoch 21/100
7200/7200 [==============================] - 3s 392us/step - loss: 0.4053 - acc: 0.8354
Epo

/anaconda/lib/python3.6/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.122547). Check your callbacks.
  % delta_t_median)


7200/7200 [==============================] - 3s 418us/step - loss: 0.3917 - acc: 0.8387
Epoch 26/100
7200/7200 [==============================] - 3s 428us/step - loss: 0.4040 - acc: 0.8342
Epoch 26/100
7200/7200 [==============================] - 3s 435us/step - loss: 0.3950 - acc: 0.8390
Epoch 27/100
7200/7200 [==============================] - 3s 462us/step - loss: 0.3962 - acc: 0.8369
Epoch 27/100
7200/7200 [==============================] - 3s 426us/step - loss: 0.3918 - acc: 0.8404
Epoch 27/100
7200/7200 [==============================] - 3s 436us/step - loss: 0.4033 - acc: 0.8354
Epoch 27/100
7200/7200 [==============================] - 3s 414us/step - loss: 0.3912 - acc: 0.8396
Epoch 28/100
7200/7200 [==============================] - 3s 428us/step - loss: 0.3960 - acc: 0.8353
Epoch 28/100
7200/7200 [==============================] - 3s 467us/step - loss: 0.3952 - acc: 0.8375
Epoch 28/100
7200/7200 [==============================] - 3s 423us/step - loss: 0.4031 - acc: 0.8354
Epo

7200/7200 [==============================] - 3s 420us/step - loss: 0.3894 - acc: 0.8406
Epoch 45/100
7200/7200 [==============================] - 3s 418us/step - loss: 0.4014 - acc: 0.8362
Epoch 45/100
7200/7200 [==============================] - 3s 429us/step - loss: 0.3924 - acc: 0.8378
Epoch 45/100
7200/7200 [==============================] - 3s 429us/step - loss: 0.3764 - acc: 0.8428
Epoch 45/100
7200/7200 [==============================] - 3s 440us/step - loss: 0.3898 - acc: 0.8401
Epoch 46/100
7200/7200 [==============================] - 3s 462us/step - loss: 0.4014 - acc: 0.8343
Epoch 46/100
7200/7200 [==============================] - 4s 486us/step - loss: 0.3920 - acc: 0.8367
Epoch 46/100
Epoch 46/100
7200/7200 [==============================] - 3s 457us/step - loss: 0.3898 - acc: 0.8418
Epoch 47/100
7200/7200 [==============================] - 3s 441us/step - loss: 0.4015 - acc: 0.8357
Epoch 47/100
7200/7200 [==============================] - 3s 450us/step - loss: 0.3916 - ac

...

7200/7200 [==============================] - 3s 384us/step - loss: 0.3483 - acc: 0.8607
Epoch 69/100
7200/7200 [==============================] - 3s 400us/step - loss: 0.4005 - acc: 0.8392
Epoch 69/100
7200/7200 [==============================] - 3s 419us/step - loss: 0.3469 - acc: 0.8596
Epoch 69/100
7200/7200 [==============================] - 3s 446us/step - loss: 0.3413 - acc: 0.8622
Epoch 69/100
7200/7200 [==============================] - 3s 445us/step - loss: 0.3482 - acc: 0.8574
Epoch 70/100
7200/7200 [==============================] - 3s 458us/step - loss: 0.4008 - acc: 0.8337
Epoch 70/100
7200/7200 [==============================] - 3s 440us/step - loss: 0.3413 - acc: 0.8593
Epoch 70/100
7200/7200 [==============================] - 3s 450us/step - loss: 0.3470 - acc: 0.8621
Epoch 71/100
7200/7200 [==============================] - 3s 435us/step - loss: 0.4006 - acc: 0.8371
Epoch 71/100
7200/7200 [==============================] - 3s 422us/step - loss: 0.3457 - acc: 0.8610
Epo

7200/7200 [==============================] - 4s 547us/step - loss: 0.4002 - acc: 0.8360
Epoch 87/100
7200/7200 [==============================] - 4s 565us/step - loss: 0.3452 - acc: 0.8614
Epoch 87/100
7200/7200 [==============================] - 4s 568us/step - loss: 0.3399 - acc: 0.8617
Epoch 87/100
7200/7200 [==============================] - 4s 567us/step - loss: 0.3451 - acc: 0.8606
Epoch 88/100
7200/7200 [==============================] - 4s 508us/step - loss: 0.4002 - acc: 0.8351
Epoch 88/100
7200/7200 [==============================] - 3s 473us/step - loss: 0.3444 - acc: 0.8599
Epoch 88/100
7200/7200 [==============================] - 3s 448us/step - loss: 0.3395 - acc: 0.8622
Epoch 88/100
7200/7200 [==============================] - 3s 465us/step - loss: 0.4007 - acc: 0.8369
Epoch 89/100
7200/7200 [==============================] - 3s 468us/step - loss: 0.3454 - acc: 0.8600
Epoch 89/100
7200/7200 [==============================] - 3s 456us/step - loss: 0.3401 - acc: 0.8618
Epo

7200/7200 [==============================] - 2s 324us/step - loss: 0.3923 - acc: 0.8301
Epoch 6/100
7200/7200 [==============================] - 2s 334us/step - loss: 0.3868 - acc: 0.8310
Epoch 6/100
7200/7200 [==============================] - 2s 336us/step - loss: 0.3846 - acc: 0.8311
Epoch 7/100
7200/7200 [==============================] - 2s 338us/step - loss: 0.3805 - acc: 0.8356
Epoch 7/100
7200/7200 [==============================] - 2s 333us/step - loss: 0.3777 - acc: 0.8385
Epoch 8/100
7200/7200 [==============================] - 2s 343us/step - loss: 0.3745 - acc: 0.8408
Epoch 8/100
7200/7200 [==============================] - 2s 347us/step - loss: 0.3731 - acc: 0.8444
Epoch 9/100
7200/7200 [==============================] - 2s 333us/step - loss: 0.3717 - acc: 0.8462
Epoch 9/100
7200/7200 [==============================] - 2s 335us/step - loss: 0.3685 - acc: 0.8471
Epoch 10/100
7200/7200 [==============================] - 2s 335us/step - loss: 0.3681 - acc: 0.8499
Epoch 10/10

7200/7200 [==============================] - 2s 328us/step - loss: 0.3440 - acc: 0.8593
Epoch 45/100
7200/7200 [==============================] - 2s 325us/step - loss: 0.3433 - acc: 0.8599
Epoch 45/100
7200/7200 [==============================] - 2s 341us/step - loss: 0.3443 - acc: 0.8583
Epoch 46/100
7200/7200 [==============================] - 2s 337us/step - loss: 0.3438 - acc: 0.8614
Epoch 46/100
7200/7200 [==============================] - 2s 329us/step - loss: 0.3433 - acc: 0.8612
Epoch 47/100
7200/7200 [==============================] - 2s 328us/step - loss: 0.3433 - acc: 0.8614
Epoch 47/100
7200/7200 [==============================] - 2s 325us/step - loss: 0.3443 - acc: 0.8600
Epoch 48/100
7200/7200 [==============================] - 2s 331us/step - loss: 0.3435 - acc: 0.8607
Epoch 48/100
7200/7200 [==============================] - 2s 328us/step - loss: 0.3438 - acc: 0.8597
Epoch 49/100
7200/7200 [==============================] - 2s 338us/step - loss: 0.3437 - acc: 0.8603
Epo

7200/7200 [==============================] - 2s 304us/step - loss: 0.3390 - acc: 0.8596
Epoch 85/100
7200/7200 [==============================] - 2s 328us/step - loss: 0.3411 - acc: 0.8606
Epoch 85/100
7200/7200 [==============================] - 2s 330us/step - loss: 0.3382 - acc: 0.8603
Epoch 86/100
7200/7200 [==============================] - 2s 335us/step - loss: 0.3409 - acc: 0.8624
Epoch 86/100
7200/7200 [==============================] - 3s 348us/step - loss: 0.3385 - acc: 0.8606
Epoch 87/100
7200/7200 [==============================] - 2s 335us/step - loss: 0.3414 - acc: 0.8604
Epoch 87/100
7200/7200 [==============================] - 2s 336us/step - loss: 0.3387 - acc: 0.8594
Epoch 88/100
7200/7200 [==============================] - 2s 321us/step - loss: 0.3375 - acc: 0.8624
Epoch 89/100
7200/7200 [==============================] - 2s 332us/step - loss: 0.3409 - acc: 0.8625
Epoch 89/100
7200/7200 [==============================] - 2s 322us/step - loss: 0.3376 - acc: 0.8618
Epo

In [ ]:
mean = accuracies.mean()
variance = accuracies.std() 
print("mean:",mean,"variance:",variance) # Low variance and low bias

After evaluating the ANN using k fold cross validation, we can see that the accuracy is not as high as we think. Therefore, in the next section we are going to improve and tune and see if we can get a better result.

## 4. Improving and tuning ANN
We are going to use dropout to prevent overfitting and grid search to tune the hyperparameters, such as batch size, optimizer, epochs, etc. 

In [8]:
from sklearn.model_selection import GridSearchCV 
from keras.layers import Dropout

In [15]:
def build_classifier(optimizer):
    # Same code from before
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [16]:
classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [25,32],
              'epochs': [100,500],
             'optimizer': ['rmsprop']
             }
grid_search = GridSearchCV(estimator = classifier,
                          param_grid = parameters,
                          scoring = 'accuracy',
                          cv = 10)
grid_search_results = grid_search.fit(X_train,y_train)

Epoch 1/100
7200/7200 [==============================] - 1s 208us/step - loss: 0.6115 - acc: 0.7950
Epoch 2/100
7200/7200 [==============================] - 1s 101us/step - loss: 0.4545 - acc: 0.7971
Epoch 3/100
7200/7200 [==============================] - 1s 102us/step - loss: 0.4356 - acc: 0.7971
Epoch 4/100
7200/7200 [==============================] - 1s 104us/step - loss: 0.4311 - acc: 0.7971
Epoch 5/100
7200/7200 [==============================] - 1s 100us/step - loss: 0.4286 - acc: 0.7971
Epoch 6/100
7200/7200 [==============================] - 1s 104us/step - loss: 0.4265 - acc: 0.7971
Epoch 7/100
7200/7200 [==============================] - 1s 107us/step - loss: 0.4241 - acc: 0.7971
Epoch 8/100
7200/7200 [==============================] - 1s 108us/step - loss: 0.4219 - acc: 0.7971
Epoch 9/100
7200/7200 [==============================] - 1s 103us/step - loss: 0.4202 - acc: 0.7971
Epoch 10/100
7200/7200 [==============================] - 1s 109us/step - loss: 0.4187 - acc: 0.8194

7200/7200 [==============================] - 1s 100us/step - loss: 0.4001 - acc: 0.8350
Epoch 83/100
7200/7200 [==============================] - 1s 97us/step - loss: 0.4002 - acc: 0.8356
Epoch 84/100
7200/7200 [==============================] - 1s 101us/step - loss: 0.3998 - acc: 0.8343
Epoch 85/100
7200/7200 [==============================] - 1s 101us/step - loss: 0.3999 - acc: 0.8346
Epoch 86/100
7200/7200 [==============================] - 1s 100us/step - loss: 0.3999 - acc: 0.8347
Epoch 87/100
7200/7200 [==============================] - 1s 101us/step - loss: 0.4002 - acc: 0.8356
Epoch 88/100
7200/7200 [==============================] - 1s 102us/step - loss: 0.3996 - acc: 0.8346
Epoch 89/100
7200/7200 [==============================] - 1s 102us/step - loss: 0.3999 - acc: 0.8347
Epoch 90/100
7200/7200 [==============================] - 1s 101us/step - loss: 0.3998 - acc: 0.8344
Epoch 91/100
7200/7200 [==============================] - 1s 95us/step - loss: 0.4000 - acc: 0.8360
Epoch

7200/7200 [==============================] - 1s 98us/step - loss: 0.3943 - acc: 0.8379
Epoch 64/100
7200/7200 [==============================] - 1s 105us/step - loss: 0.3943 - acc: 0.8354
Epoch 65/100
7200/7200 [==============================] - 1s 103us/step - loss: 0.3947 - acc: 0.8369
Epoch 66/100
7200/7200 [==============================] - 1s 103us/step - loss: 0.3943 - acc: 0.8376
Epoch 67/100
7200/7200 [==============================] - 1s 107us/step - loss: 0.3942 - acc: 0.8381
Epoch 68/100
7200/7200 [==============================] - 1s 101us/step - loss: 0.3943 - acc: 0.8371
Epoch 69/100
7200/7200 [==============================] - 1s 98us/step - loss: 0.3941 - acc: 0.8375
Epoch 70/100
7200/7200 [==============================] - 1s 97us/step - loss: 0.3939 - acc: 0.8371
Epoch 71/100
7200/7200 [==============================] - 1s 101us/step - loss: 0.3934 - acc: 0.8375
Epoch 72/100
7200/7200 [==============================] - 1s 102us/step - loss: 0.3941 - acc: 0.8382
Epoch 

7200/7200 [==============================] - 1s 94us/step - loss: 0.4033 - acc: 0.8360
Epoch 45/100
7200/7200 [==============================] - 1s 99us/step - loss: 0.4033 - acc: 0.8360
Epoch 46/100
7200/7200 [==============================] - 1s 100us/step - loss: 0.4029 - acc: 0.8361
Epoch 47/100
7200/7200 [==============================] - 1s 103us/step - loss: 0.4028 - acc: 0.8351
Epoch 48/100
7200/7200 [==============================] - 1s 101us/step - loss: 0.4030 - acc: 0.8362
Epoch 49/100
7200/7200 [==============================] - 1s 102us/step - loss: 0.4026 - acc: 0.8340
Epoch 50/100
7200/7200 [==============================] - 1s 101us/step - loss: 0.4022 - acc: 0.8356
Epoch 51/100
7200/7200 [==============================] - 1s 101us/step - loss: 0.4025 - acc: 0.8360
Epoch 52/100
7200/7200 [==============================] - 1s 104us/step - loss: 0.4019 - acc: 0.8349
Epoch 53/100
7200/7200 [==============================] - 1s 101us/step - loss: 0.4024 - acc: 0.8361
Epoch

7200/7200 [==============================] - 1s 105us/step - loss: 0.4020 - acc: 0.8361
Epoch 26/100
7200/7200 [==============================] - 1s 103us/step - loss: 0.4013 - acc: 0.8369
Epoch 27/100
7200/7200 [==============================] - 1s 104us/step - loss: 0.4008 - acc: 0.8351
Epoch 28/100
7200/7200 [==============================] - 1s 102us/step - loss: 0.4001 - acc: 0.8371
Epoch 29/100
7200/7200 [==============================] - 1s 102us/step - loss: 0.3996 - acc: 0.8360
Epoch 30/100
7200/7200 [==============================] - 1s 99us/step - loss: 0.3991 - acc: 0.8362
Epoch 31/100
7200/7200 [==============================] - 1s 103us/step - loss: 0.3987 - acc: 0.8375
Epoch 32/100
7200/7200 [==============================] - 1s 104us/step - loss: 0.3980 - acc: 0.8364
Epoch 33/100
7200/7200 [==============================] - 1s 104us/step - loss: 0.3976 - acc: 0.8378
Epoch 34/100
7200/7200 [==============================] - 1s 104us/step - loss: 0.3972 - acc: 0.8376
Epoc

...

8000/8000 [==============================] - 1s 82us/step - loss: 0.3359 - acc: 0.8620
Epoch 98/500
8000/8000 [==============================] - 1s 80us/step - loss: 0.3358 - acc: 0.8629
Epoch 99/500
8000/8000 [==============================] - 1s 81us/step - loss: 0.3360 - acc: 0.8615
Epoch 100/500
8000/8000 [==============================] - 1s 83us/step - loss: 0.3353 - acc: 0.8628
Epoch 101/500
8000/8000 [==============================] - 1s 87us/step - loss: 0.3351 - acc: 0.8619
Epoch 102/500
8000/8000 [==============================] - 1s 81us/step - loss: 0.3355 - acc: 0.8625
Epoch 103/500
8000/8000 [==============================] - 1s 82us/step - loss: 0.3358 - acc: 0.8610
Epoch 104/500
8000/8000 [==============================] - 1s 81us/step - loss: 0.3354 - acc: 0.8624
Epoch 105/500
8000/8000 [==============================] - 1s 82us/step - loss: 0.3352 - acc: 0.8631
Epoch 106/500
8000/8000 [==============================] - 1s 82us/step - loss: 0.3355 - acc: 0.8628
Epoch 

8000/8000 [==============================] - 1s 82us/step - loss: 0.3331 - acc: 0.8619
Epoch 179/500
8000/8000 [==============================] - 1s 82us/step - loss: 0.3334 - acc: 0.8632
Epoch 180/500
8000/8000 [==============================] - 1s 81us/step - loss: 0.3335 - acc: 0.8619
Epoch 181/500
8000/8000 [==============================] - 1s 82us/step - loss: 0.3331 - acc: 0.8620
Epoch 182/500
8000/8000 [==============================] - 1s 82us/step - loss: 0.3334 - acc: 0.8626
Epoch 183/500
8000/8000 [==============================] - 1s 81us/step - loss: 0.3337 - acc: 0.8630
Epoch 184/500
8000/8000 [==============================] - 1s 83us/step - loss: 0.3331 - acc: 0.8639
Epoch 185/500
8000/8000 [==============================] - 1s 83us/step - loss: 0.3323 - acc: 0.8621
Epoch 186/500
8000/8000 [==============================] - 1s 78us/step - loss: 0.3330 - acc: 0.8620
Epoch 187/500
8000/8000 [==============================] - 1s 80us/step - loss: 0.3338 - acc: 0.8624
Epoc

8000/8000 [==============================] - 1s 79us/step - loss: 0.3325 - acc: 0.8615
Epoch 260/500
8000/8000 [==============================] - 1s 79us/step - loss: 0.3333 - acc: 0.8621
Epoch 261/500
8000/8000 [==============================] - 1s 77us/step - loss: 0.3328 - acc: 0.8610
Epoch 262/500
8000/8000 [==============================] - 1s 81us/step - loss: 0.3330 - acc: 0.8638
Epoch 263/500
8000/8000 [==============================] - 1s 81us/step - loss: 0.3334 - acc: 0.8611
Epoch 264/500
8000/8000 [==============================] - 1s 82us/step - loss: 0.3327 - acc: 0.8621
Epoch 265/500
8000/8000 [==============================] - 1s 79us/step - loss: 0.3335 - acc: 0.8620
Epoch 266/500
8000/8000 [==============================] - 1s 79us/step - loss: 0.3335 - acc: 0.8609
Epoch 267/500
8000/8000 [==============================] - 1s 82us/step - loss: 0.3324 - acc: 0.8635
Epoch 268/500
8000/8000 [==============================] - 1s 83us/step - loss: 0.3331 - acc: 0.8615
Epoc

8000/8000 [==============================] - 1s 81us/step - loss: 0.3325 - acc: 0.8636
Epoch 341/500
8000/8000 [==============================] - 1s 82us/step - loss: 0.3333 - acc: 0.8628
Epoch 342/500
8000/8000 [==============================] - 1s 82us/step - loss: 0.3330 - acc: 0.8619
Epoch 343/500
8000/8000 [==============================] - 1s 82us/step - loss: 0.3326 - acc: 0.8609
Epoch 344/500
8000/8000 [==============================] - 1s 81us/step - loss: 0.3324 - acc: 0.8634
Epoch 345/500
8000/8000 [==============================] - 1s 82us/step - loss: 0.3325 - acc: 0.8646
Epoch 346/500
8000/8000 [==============================] - 1s 82us/step - loss: 0.3328 - acc: 0.8638
Epoch 347/500
8000/8000 [==============================] - 1s 82us/step - loss: 0.3331 - acc: 0.8631
Epoch 348/500
8000/8000 [==============================] - 1s 80us/step - loss: 0.3325 - acc: 0.8634
Epoch 349/500
8000/8000 [==============================] - 1s 82us/step - loss: 0.3326 - acc: 0.8634
Epoc

8000/8000 [==============================] - 1s 80us/step - loss: 0.3324 - acc: 0.8613
Epoch 422/500
8000/8000 [==============================] - 1s 76us/step - loss: 0.3334 - acc: 0.8619
Epoch 423/500
8000/8000 [==============================] - 1s 82us/step - loss: 0.3325 - acc: 0.8629
Epoch 424/500
8000/8000 [==============================] - 1s 82us/step - loss: 0.3337 - acc: 0.8618
Epoch 425/500
8000/8000 [==============================] - 1s 75us/step - loss: 0.3329 - acc: 0.8635
Epoch 426/500
8000/8000 [==============================] - 1s 81us/step - loss: 0.3322 - acc: 0.8626
Epoch 427/500
8000/8000 [==============================] - 1s 81us/step - loss: 0.3331 - acc: 0.8609
Epoch 428/500
8000/8000 [==============================] - 1s 80us/step - loss: 0.3318 - acc: 0.8634
Epoch 429/500
8000/8000 [==============================] - 1s 80us/step - loss: 0.3330 - acc: 0.8639
Epoch 430/500
8000/8000 [==============================] - 1s 80us/step - loss: 0.3324 - acc: 0.8613
Epoc

In [13]:
best_para = grid_search_results.best_params_
best_accuracy = grid_search_results.best_score_
best_para
best_accuracy

0.80412499999999998